# ABI - Versão inicial 1

(Descrição, ao finalizar)

# V1: pré-processamento de texto alternativo

## Parte 0: importação de bibliotecas

Usaremos as seguintes bibliotecas (TODO: adicionar links para a documentação):

- Numpy: para vetorização de dados;
- Pandas: para manipulação de base dados;
- Random: para números aleatórios e seed;
- Unidecode: pra processamento de acentos;
- NLTK: para ferramentas de NLP;
- Re: para processamento de regex;
- Scikitlearn: para modelagem estatística e machine learning;
- TensorFlow: framework de redes neurais e deep learning;
- Keras: API de alto nível para o TensorFlow;
- Matplotlib: para plots e visualizações
- Seaborn: para plotagem estatística

### Importando de principais bibliotecas

In [ ]:
#processamento de dados
import numpy as np
import pandas as pd

#regex
import re

#nlp
import nltk
# nltk.download('stopwords')
# nltk.download('rslp')

#dataviz
import matplotlib.pyplot as plt

#random
import random
#pra termos resultados mais fixos

#pra tirar acentos
import unidecode

## Parte 1: leitura de dados

### Lendo a base de perguntas

In [ ]:
#numero de perguntas pra cada intent
n = 80

df_q = pd.read_csv("base_treino_" + str(n) + ".csv")

#é bom dar uma misturada...
df_q = df_q.sample(frac=1, random_state = 42).reset_index(drop=True)
 
#essa é a base desbalanceada
df_desbalanc = df_q[~df_q["intent"].isin(df_q["intent"].value_counts()[df_q["intent"].value_counts() == n].index.tolist())]

#essa é a base balanceada (vou manter o mesmo nome)
df_q = df_q[df_q["intent"].isin(df_q["intent"].value_counts()[df_q["intent"].value_counts() == n].index.tolist())]
    
if (~df_q["intent"].value_counts().values == n).sum() == 0:
    print("Bases balanceada e desbalanceadas separadas com sucesso!")
    display(df_q.head())
else:
    print("Erro... Olha o código!")
    assert(False)

In [ ]:
df_q.info()

In [ ]:
perguntas = df_q.iloc[:,1]
print("\nTemos", len(perguntas), "perguntas na base.\nAs perguntas estão distribuídas em diferentes intents:\n")

intents = df_q.iloc[:,0]
print("Temos", len(intents.unique()), "intents na base, com as respectivas quantidades de perguntas",
                                      "(que idealmente devem ser balanceadas entre os intents):")
display(intents.value_counts())

In [ ]:
#############################################################################################
#############################################################################################
######################################################## PRA PARTE 2:

#pra fazer o bag of words
from sklearn.feature_extraction.text import CountVectorizer

#############################################################################################
#############################################################################################
######################################################## PRA PARTE 3:

#train-test split
from sklearn.model_selection import train_test_split

#avaliação de performanace
from sklearn.metrics import confusion_matrix, classification_report

#Modelos:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#instanciando os modelos
modelos = {"gnb": GaussianNB(),
           "mnb": MultinomialNB(),
           "logit": LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state = 42),
           "svm": SVC(decision_function_shape='ovo', gamma="auto", kernel="linear", C=10, random_state = 42),
           "rf": RandomForestClassifier(n_estimators=300, max_depth=None, random_state = 42)}

# Pipeline de pré-processamento e modelagem

## Passos 2 e 3

__Definição dos modos do pipeline__

In [ ]:
# lista de tuplas com os modos do pre-processing
# formato: (tira_numeros_string(T/F), stemmer, tira_acento_corpus(T/F), 
#           tira_acento_antes_ou_depois_do_stemmer, tira stopwords, descrição)

modos = [(True, "RSLPS", False, "-", True, "sem números, RSLPS, com acento no corpus, tirando stopwords"),
         (False, "RSLPS", False, "-", True, "com números, RSLPS, com acento no corpus, tirando stopwords"),
         (True, "SB", False, "-", True, "sem números, SB, com acento no corpus, tirando stopwords"),
         (False, "SB", False, "-", True, "com números, SB, com acento no corpus, tirando stopwords"),
         
         (True, "RSLPS", True, "antes", True, "sem números, RSLPS, sem acento no corpus, tira acento antes do stemming, tirando stopwords"),
         (False, "RSLPS", True, "antes", True, "com números, RSLPS, sem acento no corpus, tira acento antes do stemming, tirando stopwords"),
         (True, "SB", True, "antes", True, "sem números, SB, sem acento no corpus, tira acento antes do stemming, tirando stopwords"),
         (False, "SB", True, "antes", True, "com números, SB, sem acento no corpus, tira acento antes do stemming, tirando stopwords"),
         
         (True, "RSLPS", True, "depois", True, "sem números, RSLPS, sem acento no corpus, tira acento depois do stemming, tirando stopwords"),
         (False, "RSLPS", True, "depois", True, "com números, RSLPS, sem acento no corpus, tira acento depois do stemming, tirando stopwords"),
         (True, "SB", True, "depois", True, "sem números, SB, sem acento no corpus, tira acento depois do stemming, tirando stopwords"),
         (False, "SB", True, "depois", True, "com números, SB, sem acento no corpus, tira acento depois do stemming, tirando stopwords"), 
         
         
         (True, "RSLPS", False, "-", False, "sem números, RSLPS, com acento no corpus, sem tirar stopwords"),
         (False, "RSLPS", False, "-", False, "com números, RSLPS, com acento no corpus, sem tirar stopwords"),
         (True, "SB", False, "-", False, "sem números, SB, com acento no corpus, sem tirar stopwords"),
         (False, "SB", False, "-", False, "com números, SB, com acento no corpus, sem tirar stopwords"),
         
         (True, "RSLPS", True, "antes", False, "sem números, RSLPS, sem acento no corpus, tira acento antes do stemming, sem tirar stopwords"),
         (False, "RSLPS", True, "antes", False, "com números, RSLPS, sem acento no corpus, tira acento antes do stemming, sem tirar stopwords"),
         (True, "SB", True, "antes", False, "sem números, SB, sem acento no corpus, tira acento antes do stemming, sem tirar stopwords"),
         (False, "SB", True, "antes", False, "com números, SB, sem acento no corpus, tira acento antes do stemming, sem tirar stopwords"),
         
         (True, "RSLPS", True, "depois", False, "sem números, RSLPS, sem acento no corpus, tira acento depois do stemming, sem tirar stopwords"),
         (False, "RSLPS", True, "depois", False, "com números, RSLPS, sem acento no corpus, tira acento depois do stemming, sem tirar stopwords"),
         (True, "SB", True, "depois", False, "sem números, SB, sem acento no corpus, tira acento depois do stemming, sem tirar stopwords"),
         (False, "SB", True, "depois", False, "com números, SB, sem acento no corpus, tira acento depois do stemming, sem tirar stopwords")]


#pra mostrar o corpus antes e após o pré-processamento (caso True)
show_preprocess = False

#nesta lista eu vou colocar todos os modelos que eu fitar em cada modo, com as respectivas indicações
#formato da lista modelos_fitados:
#[modo, [nome_do_modelo, modelos_fitados]*numero_de_modelos]
modelos_fitados = []

#nessa lista eu vou guardar apenas os melhores modelos de cada modo
#formato da lista melhores_modelos_fitados
#[modo, [nome_do_melhor_modelo, weighted_f1_do_melhor_modelo, melhores_modelos_fitados]*1]
melhores_modelos_fitados = []

#stopwords
stpwrds = nltk.corpus.stopwords.words('portuguese')

for (idx, modo) in enumerate(modos):
    
    modelos_fitados.append(["Modo: " + modo[-1]])
    print("Modo: " + modo[-1])
    
    melhores_modelos_fitados.append(["Modo: " + modo[-1]])

    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #PASSO 2: PRÉ-PROCESSAMENTO

    # Etapas de pré-processamento (todas são discutíveis):

    # - limpeza do texto: retiramos números e pontuação;
    # - deixamos todo o texto em minúsculas;
    # - aplicamos stemming. Temos duas opções de stemmers:
    #     - RSLP Stemmer: http://www.inf.ufrgs.br/~viviane/rslp/index.htm
    #     - Snowball (Porter2): https://snowballstem.org/
    # - retiramos as stopwords das perguntas (TODO: talvez reconsiderar e incluir "não", entre outras);
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________


    #############################################################################################
    #############################################################################################
    ############################################################################################# 
    ######################################## CRIA O CORPUS

    corpus = []

    for item in perguntas:
        
        #lembrando posições das tuplas de "modos":
        #modo[0] = tira_numeros_string(T/F)
        #modo[1] = stemmer
        #modo[2] = tira_acento_corpus(T/F)
        #modo[3] = tira_acento_antes_ou_depois_do_stemmer
        #modo[4] = tira_stop_words(T/F)
        #modo[-1] = descrição
        

        #vamos jogar fora tudo que não são letras minusculas e maiusculas (incluido acentuações)
        #vou manter também números!
        
        if modo[0]:
            #joga fora os numeros tb
            pergunta = re.sub("[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]", " ", item)
        else:
            #mantem os numeros
            pergunta = re.sub("[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ0-9]", " ", item)
            

        #deixa tudo minuscula
        pergunta = pergunta.lower()

        #tonkeniza
        pergunta = pergunta.split()

        if modo[1] == "RSLPS":
            stemmer = nltk.stem.RSLPStemmer()
        elif modo[1] == "SB":
            stemmer = nltk.stem.SnowballStemmer('portuguese')
        else:
            print("Nenhum stemer foi escolhido!")
            assert(False)

        #tira as stopwords ou não
        if modo[4]:
            #vamos tirar stopwords, e já aplicar o stemmer
            if modo[2]:
                #também vou tirar todos os acentos com o unidecode
                if modo[3] == "antes":
                    pergunta = [stemmer.stem(unidecode.unidecode(word)) for word in pergunta if word not in set(stpwrds)]
                elif modo[3] == "depois":
                    pergunta = [unidecode.unidecode(stemmer.stem(word)) for word in pergunta if word not in set(stpwrds)]
            else:
                #nao tiro o acento
                pergunta = [stemmer.stem(word) for word in pergunta if word not in set(stpwrds)]
        else:
            #não tira as stopwords
            pergunta = [stemmer.stem(word) for word in pergunta]
            
        #refaz a string
        pergunta = " ".join(pergunta)

        #adiciona ao corpus
        corpus.append(pergunta)


    if show_preprocess:
#         print("\nComparação entre a pergunta original e a versão pré-processada da pergunta no corpus:\n")

#         for i in range(len(perguntas)):
#             print(perguntas.tolist()[i], "|", corpus[i])
            
        print("\nCorpus criado com sucesso!")
    else:
        print("\nCorpus criado com sucesso!")


    print("\n___________________________________________________________________")
    print("___________________________________________________________________")
    print("___________________________________________________________________\n")

    #############################################################################################
    #############################################################################################
    ############################################################################################# 
    ######################################## ANALISA O VOCABULARIO  

    #Pra começar, vamos fazer um pequeno estudo quanto ao vocabulário do Corpus

    #só pra ter uma ideia do vocabulário, vamos fazer uma lista de listas com o formato:
    #vocabulario[i] = [palavra, numero_de_aparicoes_no_corpus]

    vocabulario = []
    for pergunta in corpus:
        for palavra in pergunta.split():
            #não queremos palavras de uma única letra (pode acontecer devido ao stemming...)
            if len(palavra) > 1:
                if palavra not in [x[0] for x in vocabulario]:
                    vocabulario.append([palavra, 1])
                else:
                    vocabulario[[x[0] for x in vocabulario].index(palavra)][1] += 1

    print("\nO vocabulário é formado por", len(vocabulario), "palavras!")

    #a partir do vocabulário, crio um dataframe com a contagem
    vocab_count = pd.DataFrame({"palavra": [],
                               "count": []})

    vocab_count["palavra"] = pd.Series(vocabulario).apply(lambda x: x[0])
    vocab_count["count"] = pd.Series(vocabulario).apply(lambda x: x[1])
    vocab_count = vocab_count.sort_values("count", ascending=False)

    if show_preprocess:
        print("\nTemos a seguir as 10 mais comuns, com as respectivas contagens:")
        display(vocab_count.head(10))

    raras = vocab_count[vocab_count["count"] == 1]
    prop_raras = raras.shape[0]/vocab_count.shape[0]
    
    print("\nA quantidade de palavras com apenas uma aparição no corpus (palavras raras)\né de " + str(raras.shape[0]) +
         ", o que equivale a", str(round(100*(prop_raras), 2)) + "% da base!!")


    print("\n___________________________________________________________________")
    print("___________________________________________________________________")
    print("___________________________________________________________________\n")

    #############################################################################################
    #############################################################################################
    ############################################################################################# 
    ######################################## FAZ O BAG OF WORDS

    #Para fazer o bag of words propriamente, vamos usar o CountVectorizer do sklearn.

    #o max_feacutres imita o tamanho do vocabulario.. Não sei o quanto é interessante
    # cv = CountVectorizer(max_features = 1500)
    cv = CountVectorizer()

    #features
    X = cv.fit_transform(corpus).toarray()

    #target
    # y = intents_num
    y = intents.values

    #validação
    if X.shape[1] == vocab_count.shape[0] and X.shape[0] == len(perguntas):
        print("As dimensões da matrix de features estão corretas!")
        print("\nBag of words feito com sucesso!")
    else:
        print("As dimensões não batem! Reveja o código!")
        assert(False)


    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    # Parte 3: modelagem preditiva

    # Testaremos agora diferentes modelos para a previsão dos intents com base nas perguntas
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________

    #############################################################################################
    #############################################################################################
    ############################################################################################# 
    ######################################## TRAIN-TEST SPLIT

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

    if show_preprocess:
        print("Check de balanço do train-test split:")
        display(pd.Series(y_train).value_counts())
        display(pd.Series(y_test).value_counts())

    #############################################################################################
    #############################################################################################
    ############################################################################################# 
    ######################################## MODELOS

    #inicializa o melhor f1 como zero
    melhor_f1 = 0
    for modelo in list(modelos.keys()):
 
        if show_preprocess:
            print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

            print("Modelo:", modelo)
            print("\nParâmetros:", modelos[modelo])
        
        fit = modelos[modelo].fit(X_train, y_train)
        y_pred = fit.predict(X_test)

        if show_preprocess:
            print("\nAvaliação:\n")
            # print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred))
        else:
            print("\nModelo", modelo, "fitado com sucesso!")
        
        #salva o modelo fitado na forma modelos_fitados[i] = [nome_do_modelo, modelos_fitados]
        modelos_fitados[idx].append([modelo, fit])
        
        #vendo qual é o melhnor modelo
        cr = classification_report(y_test, y_pred, output_dict=True)
        f1 = cr["weighted avg"]["f1-score"]
        
        if f1 > melhor_f1:
            melhor_f1 = f1
            melhor = modelo
            melhor_fit = fit

    print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
    
    print("Resumo do modo atual:")
    print("Modo:", modo[-1])
    print("\nModelo com maior weighted f1:", melhor, ", com weighted f1 de", melhor_f1)
    
    print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")  
    
    melhores_modelos_fitados[idx].append([melhor, melhor_f1, modo, melhor_fit])
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    print("\n\n_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("\n\t\t\t\t\tNOVO MODO")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________")
    print("_____________________________________________________________________________________________\n\n")
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________
    #__________________________________________________________________________________________________________________

__Resumo visual da performance de cada modo__

In [ ]:
lista_melhores = [[y[0], round(y[1], 3)] for y in [x[1] for x in melhores_modelos_fitados]]
#foco nas stopwords
lista_modos = [[x[0].split(", ")[-1]] for x in melhores_modelos_fitados]

lista_resumo = []
for i in range(len(lista_melhores)):
    lista_resumo.append(lista_modos[i] + lista_melhores[i])
    
print(*lista_resumo, sep="\n")

__Conclusão 1: tem que tirar stopwords!__

In [ ]:
#agora vamos desconsiderar os "sem tirar stopwords"
lista_modos = [["|".join(x[0].split(": ")[-1].split(", ")[:-1])] for x in melhores_modelos_fitados if x[0].split(", ")[-1] != "sem tirar stopwords"]
lista_melhores = [[y[0], round(y[1],3)] for y in [x[1] for x in melhores_modelos_fitados if x[0].split(", ")[-1] != "sem tirar stopwords"]]

lista_resumo = []
for i in range(len(lista_melhores)):
    lista_resumo.append(lista_modos[i] + lista_melhores[i])
    
print(*lista_resumo, sep="\n")

__Conclusão 2: parece que deixar os números é melhor, e usando o SB, e aparentemente tanto faz tirar o acento antes ou depois do stemming, mas é bom tirar!__

______________
______________
______________
______________
______________
______________
______________
______________
______________

## Parte 4: Obot

Vamos pegar o melhor modelo dos que foram treinados acima, e usar aqui pra simular o bot

In [ ]:
#essa função, será refinada depois. Por enquanto, tô fazendo na mão uma resposta únca
def respostas_bot(x):
    
    respostas = [['Acessórios', 'Olá! O aparelho acompanha fone de ouvido e carregador.'],
                 ['Bateria', 'Olá! A bateria é de 3500mAh.'],
                 ['Capacidade', 'Olá! A capacidade é de 16Gb.'],
                 ['Cor', 'Olá! As cores disponíveis são branco e preto.'],
#                  ['Câmera', 'Olá! A câmera é de 64 MP.'],
                 ['Dimensão', 'Olá! As dimensões são 15x10x1.'],
                 ['Disponibilidade', 'Olá! Produto disponível para pronta entrega!'],
                 ['Entrega', 'Olá! Digite seu CEP no canto esquerdo da página para estas informações!'],
#                  ['Especificação', 'Olá! O aparelho conta com bluetooth!'],
                 ['Estado', 'Olá! O produto é novo.'],
                 ['Garantia', 'Olá! O produto tem 3 anos de garantia de fábrica.'],
#                  ['Marca', 'Olá! A marca é Xiaomi.'],
                 ['Meios de pagamento', 'Olá! Aceitamos boleto e cartão de crédito de todas as bandeiras.'],
#                  ['Modelo', 'Olá! O modelo é 23240-XRF.'],
                 ['Nota Fiscal', 'Olá! A nota fiscal acompanhará sim o pedido!'],
#                  ['Tela', 'Olá! A tela é amoled full HD'],
                 ['Voltagem', 'Olá! O produto tem voltagem de 110V']]
                 
    return respostas[[item[0] for item in respostas].index(x)][1]

In [ ]:
#essa é a função de pre-processamento
def pre_proc_test(pergunta, modo):
    
    #lembrando posições das tuplas de "modos":
    #modo[0] = tira_numeros_string(T/F)
    #modo[1] = stemmer
    #modo[2] = tira_acento_corpus(T/F)
    #modo[3] = tira_acento_antes_ou_depois_do_stemmer
    #modo[4] = tira_stop_words(T/F)
    #modo[-1] = descrição


    #vamos jogar fora tudo que não são letras minusculas e maiusculas (incluido acentuações)
    #vou manter também números!

    if modo[0]:
        #joga fora os numeros tb
        pergunta = re.sub("[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]", " ", pergunta)
    else:
        #mantem os numeros
        pergunta = re.sub("[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ0-9]", " ", pergunta)


    #deixa tudo minuscula
    pergunta = pergunta.lower()

    #tonkeniza
    pergunta = pergunta.split()

    if modo[1] == "RSLPS":
        stemmer = nltk.stem.RSLPStemmer()
    elif modo[1] == "SB":
        stemmer = nltk.stem.SnowballStemmer('portuguese')
    else:
        print("Nenhum stemer foi escolhido!")
        assert(False)

    #tira as stopwords ou não
    if modo[4]:
        #vamos tirar stopwords, e já aplicar o stemmer
        if modo[2]:
            #também vou tirar todos os acentos com o unidecode
            if modo[3] == "antes":
                pergunta = [stemmer.stem(unidecode.unidecode(word)) for word in pergunta if word not in set(stpwrds)]
            elif modo[3] == "depois":
                pergunta = [unidecode.unidecode(stemmer.stem(word)) for word in pergunta if word not in set(stpwrds)]
        else:
            #nao tiro o acento
            pergunta = [stemmer.stem(word) for word in pergunta if word not in set(stpwrds)]
    else:
        #não tira as stopwords
        pergunta = [stemmer.stem(word) for word in pergunta]

    #refaz a string
    pergunta = " ".join(pergunta)

    return pergunta

In [ ]:
def chat(model, modo, th1 = 2):
    
    print("Digite sua pergunta sobre o produto!")
    
    while True:
        
        pergunta = input("Pergunta: ")
        
        if pergunta.lower() == "sair":
            break
        
        #faz o pre-processing da pergunta
        pergunta = pre_proc_test(pergunta, modo)
            
        #eu tenho que fazer o bag da pergunta usando o mesmo cv e corpus de antes, claro
        X = cv.transform([pergunta]).toarray()
        
        #essas são as probabilidades das classes
        class_probs = model.predict_proba(X)
        
        #essa é a chance random
        th2 = 1/len(model.classes_)
        
#         #ele tenta responder apenas se a chance de estar certo for th vezes maior que a aleatoriedade
#         if class_probs.max() > th1*th2:
#             y_pred = model.classes_[class_probs.argmax()]
#             resposta = respostas_bot(y_pred)
# #             print(y_pred)
# #             print(class_probs)
# #             print(class_probs[abs(class_probs - np.mean(class_probs)) > 1 * np.std(class_probs)])
#         else:
#             resposta = "Eu não entendi o que você quis dizer..."  

                

        #aqui, eu pego quais são as classes que são mais probáveis
        #a forma como eu defino é: classes que têm probabilidade maior do que th1*th2 da probabilidade máxima
        #essas são as classes que o algoritmo mais acha que são as corretas
        #pode ser que seja só uma, claro
        #mas pode ser que seja mais de uma... se for esse o caso, eu dou output de duas classes
        class_max = class_probs[class_probs > class_probs.max() - th1*th2]

        resposta = []
        nao_sei = True
        for prob in class_max:
            if prob > th1*th2:
                y_pred = model.classes_[class_probs.squeeze().tolist().index(prob)]
                resposta.append(respostas_bot(y_pred))

                nao_sei = False
                
        #trasnforma as respostas em strings, separadas por "\n"
        resposta = "\n".join(resposta)
        
        #mas, se não houver nenhuma resposta muito provável...
        if nao_sei:
            resposta = "Eu não entendi o que você quis dizer..." 
        
        print(resposta)
        
        print("\n#################################\n")
        
#lista de lista dos melhores modelos
melhores_modelos_lista = [x[1] for x in melhores_modelos_fitados]

#pega o indice do melhor modelo (o com maior weighted f1)
idx = np.array([x[1] for x in [y[1] for y in melhores_modelos_fitados]]).argmax()

#seleciona o melhor modelo
melhor_modelo = melhores_modelos_lista[idx][-1]

#seleciona o modo do melhor modelo:
melhor_modo = melhores_modelos_lista[idx][2]

chat(model=melhor_modelo, modo=melhor_modo)